In [1]:
import pandas as pd

In [ ]:
def tenkan(data, t=9):
    return (((data.rolling(t).max()) + (data.rolling(t).min())) / 2).rename("tenkan")


def kijun(data, t=26):
    return (((data.rolling(t).max()) + (data.rolling(t).min())) / 2).rename("kijun")


def chikou(data, t=26):
    return (data.shift(-t)).rename("chikou")


def span_a(data, t_short=9, t_mid=26):
    return ((((tenkan(data, t_short) + kijun(data, t_mid))) / 2).shift(t_mid)).rename("span_a")


def span_b(data, t_mid=26, t_long=52):
    return ((((data.rolling(t_long).max()) + (data.rolling(t_long).min())) / 2).shift(t_long)).rename("span_b")

In [3]:
def apply_ichi(data, short=9, medium=26, long=52):
    """
    data : pandas.core.series.Series

    returns : pandas.core.frame.DataFrame
    """
    if not isinstance(data, pd.Series):
        raise TypeError(f"input data must be {pd.Series}")

    data = data.copy()

    _tenkan = tenkan(data, short)
    _kijun = kijun(data, medium)
    _chikou = chikou(data, medium)
    _span_a = span_a(data, short, medium)
    _span_b = span_b(data, medium, long)

    return pd.concat([data, _tenkan, _kijun, _chikou, _span_a, _span_b], axis=1)


In [ ]:
import pandas as pd

def calculate_ichimoku(df):
    """
    Calculate Ichimoku Cloud indicators and align them for trading signals.
    
    Args:
        df (pd.DataFrame): Input DataFrame with columns ['High', 'Low', 'Close']
        
    Returns:
        pd.DataFrame: DataFrame with Ichimoku columns added.
    """
    # Copy to avoid modifying original data
    df = df.copy()
    
    # Tenkan-sen (Conversion Line)
    period9_high = df['High'].rolling(window=9, min_periods=0).max()
    period9_low = df['Low'].rolling(window=9, min_periods=0).min()
    df['tenkan_sen'] = (period9_high + period9_low) / 2
    
    # Kijun-sen (Base Line)
    period26_high = df['High'].rolling(window=26, min_periods=0).max()
    period26_low = df['Low'].rolling(window=26, min_periods=0).min()
    df['kijun_sen'] = (period26_high + period26_low) / 2
    
    # Senkou Span A (Leading Span A)
    df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26)
    
    # Senkou Span B (Leading Span B)
    period52_high = df['High'].rolling(window=52, min_periods=0).max()
    period52_low = df['Low'].rolling(window=52, min_periods=0).min()
    df['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(26)
    
    # Chikou Span (Lagging Span)
    df['chikou_span'] = df['Close'].shift(-26)
    
    return df

# Example usage:
# Assuming `df` is your DataFrame with ['High', 'Low', 'Close'] columns
# df_ichimoku = calculate_ichimoku(df)

# Example usage with sample/generated data:
if __name__ == "__main__":
    # --- (Replace this section with your own OHLC DataFrame) ---
    import numpy as np
    
    data = pd.read_csv("data2.csv")
    returns = np.log(data["Close"] / data["Close"].shift(1)).dropna()
    # ----------------------------------------------------------
    
    ichimoku_df = apply_ichi(returns)
    
    # Show the last few rows (so you can see that Tenkan, Kijun, SenkouA, SenkouB, and Chikou are all populated)
    print(ichimoku_df.tail(50))


ValueError: window must be an integer 0 or greater